<a href="https://colab.research.google.com/github/rgvictor03/rankqa/blob/TdL/abstractive_question_answering_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Abstractive Question Answering

Abstractive question-answering focuses on the generation of multi-sentence answers to open-ended questions. It usually works by searching massive document stores for relevant information and then using this information to synthetically generate answers. This notebook demonstrates how Pinecone helps you build an abstractive question-answering system. We need three main components:

- A vector index to store and run semantic search
- A retriever model for embedding context passages
- A generator model to generate answers

# Install Dependencies

In [1]:
!pip install -qU datasets pinecone-client sentence-transformers torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.2/177.2 kB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 8.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.9/619.9 MB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 23.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 66.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 25.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 283.7/283.7 kB 33.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 106.2 MB/s eta 

# Load and Prepare Dataset

Our source data will be taken from the Wiki Snippets dataset, which contains over 17 million passages from Wikipedia. But, since indexing the entire dataset may take some time, we will only utilize 50,000 passages in this demo that include "History" in the "section title" column. If you want, you may utilize the complete dataset. Pinecone vector database can effortlessly manage millions of documents for you.

In [2]:
from datasets import load_dataset

# load the dataset from huggingface in streaming mode and shuffle it
wiki_data = load_dataset(
    'vblagoje/wikipedia_snippets_streamed',
    split='train',
    streaming=True
).shuffle(seed=960)

We are loading the dataset in the streaming mode so that we don't have to wait for the whole dataset to download (which is over 9GB). Instead, we iteratively download records one at a time.

In [3]:
# show the contents of a single document in the dataset
next(iter(wiki_data))

{'wiki_id': 'Q7649565',
 'start_paragraph': 20,
 'start_character': 272,
 'end_paragraph': 24,
 'end_character': 380,
 'article_title': 'Sustainable Agriculture Research and Education',
 'section_title': "2000s & Evaluation of the program's effectiveness",
 'passage_text': "preserving the surrounding prairies. It ran until March 31, 2001.\nIn 2008, SARE celebrated its 20th anniversary. To that date, the program had funded 3,700 projects and was operating with an annual budget of approximately $19 million. Evaluation of the program's effectiveness As of 2008, 64% of farmers who had received SARE grants stated that they had been able to earn increased profits as a result of the funding they received and utilization of sustainable agriculture methods. Additionally, 79% of grantees said that they had experienced a significant improvement in soil quality though the environmentally friendly, sustainable methods that they were"}

In [4]:
# filter only documents with History as section_title
history = wiki_data.filter(
    lambda d: d['section_title'].startswith('History')
)

Let's iterate through the dataset and apply our filter to select the 50,000 historical passages. We will extract `article_title`, `section_title` and `passage_text` from each document.

In [5]:
from tqdm.auto import tqdm  # progress bar

total_doc_count = 50000

counter = 0
docs = []
# iterate through the dataset and apply our filter
for d in tqdm(history, total=total_doc_count):
    # extract the fields we need
    doc = {
        "article_title": d["article_title"],
        "section_title": d["section_title"],
        "passage_text": d["passage_text"]
    }
    # add the dict containing fields we need to docs list
    docs.append(doc)

    # stop iteration once we reach 50k
    if counter == total_doc_count:
        break

    # increase the counter on every iteration
    counter += 1

  0%|          | 0/50000 [00:00<?, ?it/s]

In [6]:
import pandas as pd

# create a pandas dataframe with the documents we extracted
df = pd.DataFrame(docs)
df.head()

,article_title,section_title,passage_text
0,Taupo District,History,was not until the 1950s that the region starte...
1,Sutarfeni,History & Western asian analogues,Sutarfeni History strand-like pheni were Phena...
2,The Bishop Wand Church of England School,History,The Bishop Wand Church of England School Histo...
3,Teufelsmoor,History & Situation today,"made to preserve the original landscape, altho..."
4,Surface Hill Uniting Church,History,in perpetual reminder that work and worship go...


# Initialize Pinecone Index

The Pinecone index stores vector representations of our historical passages which we can retrieve later using another vector (query vector). To build our vector index, we must first establish a connection with Pinecone. For this, we need an API from Pinecone. You can get one for free from [here](https://app.pinecone.io/), and after that, we initialize the connection as follows:

In [7]:
import pinecone

# connect to pinecone environment
pinecone.init(
    api_key="e32bb019-f167-43af-b057-ec6a26b1bee5",
    environment="asia-southeast1-gcp-free"  # find next to API key in console
)

Now we create a new index. We will name it "abstractive-question-answering" — you can name it anything we want. We specify the metric type as "cosine" and dimension as 768 because the retriever we use to generate context embeddings is optimized for cosine similarity and outputs 768-dimension vectors.

In [8]:
index_name = "abstractive-question-answering"

# check if the abstractive-question-answering index exists
if index_name not in pinecone.list_indexes():
    # create the index if it does not exist
    pinecone.create_index(
        index_name,
        dimension=768,
        metric="cosine"
    )

# connect to abstractive-question-answering index we created
index = pinecone.Index(index_name)

# Initialize Retriever

Next, we need to initialize our retriever. The retriever will mainly do two things:

- Generate embeddings for all historical passages (context vectors/embeddings)
- Generate embeddings for our questions (query vector/embedding)

The retriever will create embeddings such that the questions and passages that hold the answers to our queries are close to one another in the vector space. We will use a SentenceTransformer model based on Microsoft's MPNet as our retriever. This model performs quite well for comparing the similarity between queries and documents. We can use Cosine Similarity to compute the similarity between query and context vectors generated by this model (Pinecone automatically does this for us).

In [9]:
import torch
from sentence_transformers import SentenceTransformer

# set device to GPU if available
device = 'cuda' #if torch.cuda.is_available() else 'cpu'
# load the retriever model from huggingface model hub
retriever = SentenceTransformer("flax-sentence-embeddings/all_datasets_v3_mpnet-base", device=device)
retriever

SentenceTransformer(
  (0): Transformer({'max_seq_length': 128, 'do_lower_case': False}) with Transformer model: MPNetModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
)

# Generate Embeddings and Upsert

Next, we need to generate embeddings for the context passages. We will do this in batches to help us more quickly generate embeddings and upload them to the Pinecone index. When passing the documents to Pinecone, we need an id (a unique value), context embedding, and metadata for each document representing context passages in the dataset. The metadata is a dictionary containing data relevant to our embeddings, such as the article title, section title, passage text, etc.

In [10]:
# we will use batches of 64
batch_size = 64

for i in tqdm(range(0, len(df), batch_size)):
    # find end of batch
    i_end = min(i+batch_size, len(df))
    # extract batch
    batch = df.iloc[i:i_end]
    # generate embeddings for batch
    emb = retriever.encode(batch["passage_text"].tolist()).tolist()
    # get metadata
    meta = batch.to_dict(orient="records")
    # create unique IDs
    ids = [f"{idx}" for idx in range(i, i_end)]
    # add all to upsert list
    to_upsert = list(zip(ids, emb, meta))
    # upsert/insert these records to pinecone
    _ = index.upsert(vectors=to_upsert)

# check that we have all vectors in index
index.describe_index_stats()

  0%|          | 0/782 [00:00<?, ?it/s]

{'dimension': 768,
 'index_fullness': 0.4,
 'namespaces': {'': {'vector_count': 50001}},
 'total_vector_count': 50001}

# Initialize Generator

We will use ELI5 BART for the generator which is a Sequence-To-Sequence model trained using the ‘Explain Like I’m 5’ (ELI5) dataset. Sequence-To-Sequence models can take a text sequence as input and produce a different text sequence as output.

The input to the ELI5 BART model is a single string which is a concatenation of the query and the relevant documents providing the context for the answer. The documents are separated by a special token &lt;P>, so the input string will look as follows:

>question: What is a sonic boom? context: &lt;P> A sonic boom is a sound associated with shock waves created when an object travels through the air faster than the speed of sound. &lt;P> Sonic booms generate enormous amounts of sound energy, sounding similar to an explosion or a thunderclap to the human ear. &lt;P> Sonic booms due to large supersonic aircraft can be particularly loud and startling, tend to awaken people, and may cause minor damage to some structures. This led to prohibition of routine supersonic flight overland.

More detail on how the ELI5 dataset was built is available [here](https://arxiv.org/abs/1907.09190) and how ELI5 BART model was trained is available [here](https://yjernite.github.io/lfqa.html).

Let's initialize the BART model using transformers.

In [11]:
from transformers import BartTokenizer, BartForConditionalGeneration

# load bart tokenizer and model from huggingface
tokenizer = BartTokenizer.from_pretrained('vblagoje/bart_lfqa')
generator = BartForConditionalGeneration.from_pretrained('vblagoje/bart_lfqa')#.to(device)

All the components of our abstract QA system are complete and ready to be queried. But first, let's write some helper functions to retrieve context passages from Pinecone index and to format the query in the way the generator expects the input.

In [12]:
def query_pinecone(query, top_k):
    # generate embeddings for the query
    xq = retriever.encode([query]).tolist()
    # search pinecone index for context passage with the answer
    xc = index.query(xq, top_k=top_k, include_metadata=True)
    return xc

In [13]:
def format_query(query, context):
    # extract passage_text from Pinecone search result and add the <P> tag
    context = [f"<P> {m['metadata']['passage_text']}" for m in context]
    # concatinate all context passages
    context = " ".join(context)
    # contcatinate the query and context passages
    query = f"question: {query} context: {context}"
    return query

Let's test the helper functions. We will query the Pinecone index function we created earlier with the `query_pinecone` to get context passages and pass them to the `format_query` function.

In [14]:
query = "when was the first electric power system built?"
result = query_pinecone(query, top_k=1)
result

{'matches': [{'id': '3593',
              'metadata': {'article_title': 'Electric power system',
                           'passage_text': 'Electric power system History In '
                                           '1881, two electricians built the '
                                           "world's first power system at "
                                           'Godalming in England. It was '
                                           'powered by two waterwheels and '
                                           'produced an alternating current '
                                           'that in turn supplied seven '
                                           'Siemens arc lamps at 250 volts and '
                                           '34 incandescent lamps at 40 volts. '
                                           'However, supply to the lamps was '
                                           'intermittent and in 1882 Thomas '
                                           'Ed

In [15]:
from pprint import pprint

In [16]:
# format the query in the form generator expects the input
query = format_query(query, result["matches"])
pprint(query)

('question: when was the first electric power system built? context: <P> '
 "Electric power system History In 1881, two electricians built the world's "
 'first power system at Godalming in England. It was powered by two '
 'waterwheels and produced an alternating current that in turn supplied seven '
 'Siemens arc lamps at 250 volts and 34 incandescent lamps at 40 volts. '
 'However, supply to the lamps was intermittent and in 1882 Thomas Edison and '
 'his company, The Edison Electric Light Company, developed the first '
 'steam-powered electric power station on Pearl Street in New York City. The '
 'Pearl Street Station initially powered around 3,000 lamps for 59 customers. '
 'The power station generated direct current and')


The output looks great. Now let's write a function to generate answers.

In [17]:
def generate_answer(query):
    # tokenize the query to get input_ids
    inputs = tokenizer([query], max_length=1024, return_tensors="pt")
    # use generator to predict output ids
    ids = generator.generate(inputs["input_ids"], num_beams=2, min_length=20, max_length=40)
    # use tokenizer to decode the output ids
    answer = tokenizer.batch_decode(ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
    return pprint(answer)

In [18]:
generate_answer(query)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


('The first electric power system was built in 1881 at Godalming in England. '
 'It was powered by two waterwheels and produced alternating current that in '
 'turn supplied seven Siemens arc lamps')


As we can see, the generator used the provided context to answer our question. Let's run some more queries.

In [19]:
query = "How was the first wireless message sent?"
context = query_pinecone(query, top_k=5)
query = format_query(query, context["matches"])
generate_answer(query)

('The first wireless message was sent in 1866 by Mahlon Loomis, who had a kite '
 'on a mountaintop 14 miles apart. The kite was connected to a cable')


To confirm that this answer is correct, we can check the contexts used to generate the answer.

In [20]:
for doc in context["matches"]:
    print(doc["metadata"]["passage_text"], end='\n---\n')

by electrostatic induction or electromagnetic induction, which had too short a range to be practical. In 1866 Mahlon Loomis claimed to have transmitted an electrical signal through the atmosphere between two 600 foot wires held aloft by kites on mountaintops 14 miles apart. Thomas Edison had come close to discovering radio in 1875; he had generated and detected radio waves which he called "etheric currents" experimenting with high-voltage spark circuits, but due to lack of time did not pursue the matter. David Edward Hughes in 1879 had also stumbled on radio wave transmission which he received with his carbon microphone
---
the east coast of India, then on to Penang, Malacca, Singapore, Batavia (current Jakarta), to finally reach Darwin, Australia. It was the first direct link between Australia and Great Britain. The company that laid the first part of the cable took the name of Falmouth, Gibraltar and Malta Telegraph Company and had been founded in 1869. This company later operated as

In this case, the answer looks correct. If we ask a question and no relevant contexts are retrieved, the generator will typically return nonsensical or false answers, like with this question about COVID-19:

In [21]:
query = "where did COVID-19 originate?"
context = query_pinecone(query, top_k=3)
query = format_query(query, context["matches"])
generate_answer(query)

('COVID-19 is a zoonotic disease, which means that it is a virus that is '
 'transmitted from one animal to another. It is not a virus that can be '
 'transmitted from person')


In [22]:
for doc in context["matches"]:
    print(doc["metadata"]["passage_text"], end='\n---\n')

to establish with certainty which diseases jumped from other animals to humans, but there is increasing evidence from DNA and RNA sequencing, that measles, smallpox, influenza, HIV, and diphtheria came to humans this way. Various forms of the common cold and tuberculosis also are adaptations of strains originating in other species.
Zoonoses are of interest because they are often previously unrecognized diseases or have increased virulence in populations lacking immunity. The West Nile virus appeared in the United States in 1999 in the New York City area, and moved through the country in the summer of 2002, causing much distress. Bubonic
---
plague is a zoonotic disease, as are salmonellosis, Rocky Mountain spotted fever, and Lyme disease.
A major factor contributing to the appearance of new zoonotic pathogens in human populations is increased contact between humans and wildlife. This can be caused either by encroachment of human activity into wilderness areas or by movement of wild ani

Let’s finish with a final few questions.

In [23]:
query = "what was the war of currents?"
context = query_pinecone(query, top_k=5)
query = format_query(query, context["matches"])
generate_answer(query)

('The War of Currents was a series of events in the early 1900s between Edison '
 'and Westinghouse. The two companies were competing for the market share of '
 'electric power in the United States')


In [31]:
query = "who was the first person on the moon?"
context = query_pinecone(query, top_k=10)
query = format_query(query, context["matches"])
generate_answer(query)

('The first person to walk on the moon was Neil Armstrong in 1969. He walked '
 'on the moon in 1969. He was the first person to walk on the moon.')


In [25]:
query = "what was NASAs most expensive project?"
context = query_pinecone(query, top_k=3)
query = format_query(query, context["matches"])
generate_answer(query)

('The Space Shuttle was the most expensive project in the history of NASA. It '
 'cost about $10 billion to build.')


As we can see, the model can generate some decent answers.

In [26]:
query = "how old was Einstein when he died?"
context = query_pinecone(query, top_k=3)
query = format_query(query, context["matches"])
generate_answer(query)

('Einstein was born on January 1, 1915. He died on January 13, 1953. He was '
 'born in New York City, and lived in the United States until his death. He '
 'was')


In [27]:
for doc in context["matches"]:
    print(doc["metadata"]["passage_text"], end='\n---\n')

and while in his Western home, was a student of history, philosophy, mathematics, ethics and music. He was a find musician, being a great lover of the flute and violin, both of which he played well until he lost part of his fingers by freezing. He died August 22, 1846, in his 74th year.
---
which dynamite is the most famous.
In 1888, Nobel was astonished to read his own obituary, titled The merchant of death is dead, in a French newspaper. It was Alfred's brother Ludvig who had died; the obituary was eight years premature. The article disconcerted Nobel and made him apprehensive about how he would be remembered. This inspired him to change his will. On 10 December 1896, Alfred Nobel died in his villa in San Remo, Italy, from a cerebral haemorrhage. He was 63 years old.
Nobel wrote several wills during his lifetime. He composed the last over a year before he died,
---
until his retirement in 1905. E.J. Cooper died in 1863, but the observatory remained active until the death of Edward He

In [28]:
query = "who was Einstein?"
context = query_pinecone(query, top_k=3)
query = format_query(query, context["matches"])
generate_answer(query)

('Einstein was a German physicist who came up with the theory of relativity in '
 'the early 20th century. He was the first person to come up with the theory '
 'of relativity, which is the')


In [29]:
for doc in context["matches"]:
    print(doc["metadata"]["passage_text"], end='\n---\n')

among its students Nicolaus Copernicus, the Renaissance astronomer and polymath who would revolutionize European ideas about the universe.
---
a plenic interpretation of atomism to eliminate the void, which he considered incompatible with his concept of space. The theory was rejected by later scientists, such as Galileo's pupil Evangelista Torricelli who repeated his experiment with mercury. Blaise Pascal successfully repeated Galileo's and Torricelli's experiment and foresaw no reason why a perfect vacuum could not be achieved in principle. Scottish philosopher Thomas Carlyle mentioned Pascal's experiment in the Edinburgh Encyclopedia in an 1823 article titled "Pascal". The Magdeburg hemispheres used by Otto von Guericke in 1650 were seen by some as proof that the theory was not correct.
James Clerk Maxwell
---
detector, however he was persuaded that what he observed was induction. Neither of these individuals are usually credited with the discovery of radio, because they did not unde

In [30]:
from sentence_transformers import SentenceTransformer, util
sentences = ["Einstein was a German physicist who came up with the theory of relativity in the early 20th century. He was the first person to come up with the theory of relativity", 
             "among its students Nicolaus Copernicus, the Renaissance astronomer and polymath who would revolutionize European ideas about the universe.",
             "a plenic interpretation of atomism to eliminate the void, which he considered incompatible with his concept of space. The theory was rejected by later scientists, such as Galileo's pupil Evangelista Torricelli who repeated his experiment with mercury. Blaise Pascal successfully repeated Galileo's and Torricelli's experiment and foresaw no reason why a perfect vacuum could not be achieved in principle. Scottish philosopher Thomas Carlyle mentioned Pascal's experiment in the Edinburgh Encyclopedia in an 1823 article titled Pascal. The Magdeburg hemispheres used by Otto von Guericke in 1650 were seen by some as proof that the theory was not correct James Clerk Maxwell",
             "detector, however he was persuaded that what he observed was induction. Neither of these individuals are usually credited with the discovery of radio, because they did not understand the significance of their observations and did not publish their work before Hertz. The other was research by physicists to confirm the theory of electromagnetism proposed in 1864 by Scottish physicist James Clerk Maxwell, now called Maxwell's equations. Maxwell's theory showed that a combination of oscillating electric and magnetic fields could travel through space as an electromagnetic wave. Maxwell proposed that light consisted of electromagnetic waves of short wavelength, but no one knew"]

model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

#Compute embedding for both lists
embedding_0= model.encode(sentences[0], convert_to_tensor=True)
embedding_1 = model.encode(sentences[1], convert_to_tensor=True)
embedding_2 = model.encode(sentences[2], convert_to_tensor=True)
embedding_3 = model.encode(sentences[3], convert_to_tensor=True)

print(util.pytorch_cos_sim(embedding_0, embedding_1))
print(util.pytorch_cos_sim(embedding_0, embedding_2))
print(util.pytorch_cos_sim(embedding_0, embedding_3))
## tensor([[0.6003]])

# threshold == 0.5

tensor([[0.4868]], device='cuda:0')
tensor([[0.3417]], device='cuda:0')
tensor([[0.4870]], device='cuda:0')


In [47]:
query = "What is Phenakas?"
context = query_pinecone(query, top_k=3)
query = format_query(query, context["matches"])
answer = generate_answer(query)
answer

('Phenakas is a type of fried dough made from wheat flour and sugar. It is a '
 'popular dish in India, and it is made with a lot of sugar and flour. It')


In [48]:
context_sentences = []
for doc in context["matches"]:
    context_sentences.append(doc["metadata"]["passage_text"])
    print(doc["metadata"]["passage_text"], end='\n---\n')

Sutarfeni History strand-like pheni were Phenakas mentioned in various indian texts. Phenakas is a broad term which includes various dishes prepared by using layered fried dough. Vijayanagar records indicate that Pheni was another much relished sweet dish prepared from wheat flour and sugar, similar to phenaka of North India and had varieties like sugar pheni, milk pheni and vermicelli pheni Western asian analogues Sutarpheni is of the Indian analogs of the Turkish pismaniye, which uses wheat flour instead of rice flour, and the Persian pashmak, which substitutes sesame paste for wheat flour.  The choice of rice flour as the
---
grown into a variety of plants.
---
fossil as a new species. As was the case with most early pterosaur finds, Buckland classified the remains in the genus Pterodactylus, coining the new species Pterodactylus macronyx. The specific name is derived from Greek makros, "large" and onyx, "claw", in reference to the large claws of the hand. The specimen, presently NH

In [49]:
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

#Compute embedding for both lists
answer_embedding = model.encode(sentences[0], convert_to_tensor=True)
context_embedding_1 = model.encode(context_sentences[0], convert_to_tensor=True)
context_embedding_2 = model.encode(context_sentences[1], convert_to_tensor=True)
context_embedding_3 = model.encode(context_sentences[2], convert_to_tensor=True)

print(util.pytorch_cos_sim(answer_embedding, context_embedding_1))
print(util.pytorch_cos_sim(answer_embedding, context_embedding_2))
print(util.pytorch_cos_sim(answer_embedding, context_embedding_3))
## tensor([[0.6003]])

# threshold == 0.5

tensor([[-0.0326]], device='cuda:0')
tensor([[-0.0411]], device='cuda:0')
tensor([[-0.0175]], device='cuda:0')


# Example Application

To try out an application like this one, see this [example application](https://huggingface.co/spaces/pinecone/abstractive-question-answering).